In [6]:
import os
import sys
import re

STAGE_DIR = 'data/staging'
LOG_DIR = 'data/rab/logs'

In [16]:
edit_ptn = re.compile(
    '^[A-Z]+ '
    '(?P<date>\d{4}-\d{1,2}-\d{1,2}) (?P<time>\d{2}:\d{2}:\d{2}),.+'
    'Authenticated user: (?P<auth_user>[\w]{2,})\..+'
    'Local name: (?P<local_name>\w{2,})\.')

sesh_ptn = re.compile(
    '^[A-Z]+ '
    '(?P<date>\d{4}-\d{1,2}-\d{1,2}) (?P<time>\d{2}:\d{2}:\d{2}),.+'
    'Session created for (?P<session>\w{2,})$')

In [18]:
with open(os.path.join(STAGE_DIR,'logs','vmgr_edits.csv'), 'w') as f_edits, \
    open(os.path.join(STAGE_DIR,'logs','vmgr_sessions.csv'), 'w') as f_sesh, \
    open(os.path.join(STAGE_DIR,'logs','vmgr_log_errs.txt'), 'w') as f_errs, \
    open(os.path.join(LOG_DIR,'vivo_all.log'), 'r') as f_log:
    
    while True:
        line = f_log.readline()
        if not line:
            break
        if 'Authenticated user:' in line:
            m = re.match(edit_ptn, line)
        elif 'Session created' in line:
            m = re.match(sesh_ptn, line)
        else:
            continue
        if not m:
            f_errs.write(line)
            continue
        
        matches = m.groupdict()
        if matches.get('session'):
            f_sesh.write(
                '{},{},{}\n'.format(matches['date'],matches['time'], matches['session']))
        elif matches.get('local_name') != 'None':
            f_edits.write(
                '{},{},{}\n'.format(matches['date'],matches['time'],
                    matches['auth_user'], matches['local_name']))